In [8]:
# 시각화 라이브러리
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import math
from IPython.display import display

In [9]:
MODEL_PATH = 'models/'

# 한글 깨짐 방지
plt.rc('font', family='Malgun Gothic')

# 마이너스 기호가 깨지는 것을 방지합니다.
plt.rcParams['axes.unicode_minus'] = False

In [10]:
# 서브레딧 및 그룹 정의
SUB_PER_GROUP = 4
SUBREDDITS = [
    'Thetruthishere', 'Glitch_in_the_Matrix', 'UnresolvedMysteries', 'Paranormal',
    'learnprogramming', 'cscareerquestions', 'SideProject', 'AskProgramming',
    'TrueFilm', 'booksuggestions', 'TrueGaming', 'LetsTalkMusic',
    'relationship_advice', 'AmItheAsshole', 'offmychest', 'Advice',
    'personalfinance', 'investing', 'Frugal', 'financialindependence',
]
GROUP_MAP = {
    'Mystery': SUBREDDITS[0:SUB_PER_GROUP], 
    'Dev': SUBREDDITS[SUB_PER_GROUP:2*SUB_PER_GROUP], 
    'Culture': SUBREDDITS[2*SUB_PER_GROUP:3*SUB_PER_GROUP], 
    'Life': SUBREDDITS[3*SUB_PER_GROUP:4*SUB_PER_GROUP],
    'Finance': SUBREDDITS[4*SUB_PER_GROUP:5*SUB_PER_GROUP],
}

REVERSES_GROUP_MAP = {subreddit: group for group, subreddits in GROUP_MAP.items() for subreddit in subreddits}
VECTOR_DIMENSION = 5000  # 문서 벡터 차원

In [11]:
# 계층형 모델 로드
import joblib

# 상위 분류기 로드
top_classifier = joblib.load(MODEL_PATH + 'top_classifier.pkl')

# 하위 분류기 로드
sub_classifiers = {
    SUBREDDITS[i]: joblib.load(MODEL_PATH + f'sub_classifier_{REVERSES_GROUP_MAP[SUBREDDITS[i]]}.pkl') for i in range(len(SUBREDDITS))
}

In [12]:
# 예시 입력
example_title = "How to build a web app using Python and JavaScript"
example_input = "I want to learn how to build web applications using Python and JavaScript."

# 예측 수행
# 예측 시에는 학습에 사용된 Vectorizer를 그대로 사용해야 합니다.
# 새로운 Vectorizer를 생성하고 fit_transform을 호출하면 특징(feature)의 수가 달라져 오류가 발생합니다.
vectorizer = joblib.load(MODEL_PATH + 'tfidf_vectorizer.pkl') # 학습된 Vectorizer 로드
vectorized_input = vectorizer.transform([example_input]).toarray() # transform 메서드 사용

top_prob = top_classifier.predict_proba(vectorized_input)[0]
top_prob = {group: top_prob[i] for i, group in enumerate(top_classifier.classes_)}
predicted_group = max(top_prob, key=top_prob.get)

sub_classifier = sub_classifiers[GROUP_MAP[predicted_group][0]]  # 해당 그룹의 첫 번째 서브레딧 분류기 사용
sub_prob = sub_classifier.predict_proba(vectorized_input)[0]
sub_prob = {sub: sub_prob[i] for i, sub in enumerate(sub_classifier.classes_)}

predicted_sub = max(sub_prob, key=sub_prob.get)

print(f"Predicted Group: {predicted_group}")
print(f"Predicted Group Probabilities: {top_prob}")
print(f"Predicted Subreddit: {predicted_sub}")
print(f"Predicted Subreddit Probabilities: {sub_prob}")

Predicted Group: Dev
Predicted Group Probabilities: {np.str_('Culture'): np.float64(0.0004989070303777329), np.str_('Dev'): np.float64(0.9894887712770661), np.str_('Finance'): np.float64(0.0015495583762685503), np.str_('Life'): np.float64(0.005491860805014392), np.str_('Mystery'): np.float64(0.0029709025112733393)}
Predicted Subreddit: learnprogramming
Predicted Subreddit Probabilities: {np.str_('AskProgramming'): np.float64(0.46535708932896863), np.str_('SideProject'): np.float64(0.012295565412151916), np.str_('cscareerquestions'): np.float64(0.01794635820762805), np.str_('learnprogramming'): np.float64(0.5044009870512514)}


In [13]:
all_sub_probs = {} # {Group: {Subreddit: Prob}}
all_sub_probs[predicted_group] = sub_prob # 이미 계산된 예측 그룹 확률 저장

for group_name in GROUP_MAP.keys():
    if group_name == predicted_group:
        continue
        
    # 해당 그룹의 분류기 가져오기 (해당 그룹의 아무 서브레딧 키를 사용해 모델 로드)
    # sub_classifiers는 모든 서브레딧 키를 가지고 있으므로 첫번째 것 사용
    first_sub_key = GROUP_MAP[group_name][0]
    clf = sub_classifiers[first_sub_key]
    
    # 확률 예측
    probs_array = clf.predict_proba(vectorized_input)[0]
    probs_dict = {sub: probs_array[i] for i, sub in enumerate(clf.classes_)}
    all_sub_probs[group_name] = probs_dict

def normalize_probs(prob_dict, exclude_key=None):
    # 1등 제외하고 추출
    targets = {k: v for k, v in prob_dict.items() if k != exclude_key}
    
    if not targets: return {} # 항목이 하나뿐인 경우
    
    vals = list(targets.values())
    min_v, max_v = min(vals), max(vals)
    
    norm_dict = {}
    for k, v in targets.items():
        if max_v - min_v == 0:
            norm_dict[k] = 0.5
        else:
            # 0.0(가장 멂) ~ 1.0(가장 가까움)
            norm_dict[k] = (v - min_v) / (max_v - min_v)
            
    # 1등은 시각화를 위해 1.2 (가장 큼/가까움) 할당
    if exclude_key:
        norm_dict[exclude_key] = 1.2
        
    return norm_dict

norm_top_prob = normalize_probs(top_prob, exclude_key=predicted_group)

norm_all_sub_probs = {}
for g_name, s_probs in all_sub_probs.items():
    # 예측 그룹이면 예측 서브레딧을 제외하고 정규화
    target = predicted_sub if g_name == predicted_group else None
    norm_all_sub_probs[g_name] = normalize_probs(s_probs, exclude_key=target)

In [ ]:
# 예측 확률 시각화
def draw_network(expand_groups=[]):
    G = nx.Graph()
    pos = {}
    colors = []
    sizes = []
    labels = {}
    
    # 중심 노드 (예측된 최적 서브레딧)
    G.add_node(predicted_sub)
    pos[predicted_sub] = np.array([0, 0])
    colors.append('#FFD700') # 노란색
    sizes.append(4000)
    labels[predicted_sub] = predicted_sub.replace('r/', '')

    # 같은 그룹 내 서브레딧 (하늘색)
    siblings = norm_all_sub_probs[predicted_group]
    sibling_list = [s for s in siblings.keys() if s != predicted_sub]
    
    if sibling_list:
        angle_step = 2 * np.pi / len(sibling_list)
        for i, sub in enumerate(sibling_list):
            # 정규화된 확률 (0~1)
            n_prob = siblings[sub] 
            
            # 거리 계산: 확률이 낮을수록 멀어짐 (제곱을 써서 차이 증폭)
            # 기본거리 2.0 + (1-확률)^2 * 5.0
            dist = 2.0 + math.pow((1.0 - n_prob), 2) * 5.0
            
            angle = i * angle_step
            pos[sub] = np.array([dist * np.cos(angle), dist * np.sin(angle)])
            
            G.add_node(sub)
            colors.append('#87CEEB') # 하늘색
            sizes.append(1500 + n_prob * 1000)
            
            # 원본 확률 표시
            raw_p = all_sub_probs[predicted_group][sub]
            labels[sub] = f"{sub}\n({raw_p*100:.1f}%)"
            
            G.add_edge(predicted_sub, sub, color='gray')

    # 다른 그룹 (연두색)
    other_groups = [g for g in GROUP_MAP.keys() if g != predicted_group]
    # 상위 N개 그룹 정렬 후 슬라이싱
    other_groups = sorted(other_groups, key=lambda g: top_prob[g], reverse=True)[:2]
    
    grp_angle_step = 2 * np.pi / len(other_groups)
    
    for i, g_name in enumerate(other_groups):
        n_g_prob = norm_top_prob[g_name]
        
        # 그룹 거리: 기본 10.0 + (1-확률)^2 * 8.0 (훨씬 멀리 배치)
        base_dist = 10.0 + math.pow((1.0 - n_g_prob), 2) * 8.0
        base_angle = i * grp_angle_step + np.pi/4
        
        center_pos = np.array([base_dist * np.cos(base_angle), base_dist * np.sin(base_angle)])
        
        if g_name in expand_groups:
            # 확장 모드: 그룹 내 서브레딧들 배치
            subs = norm_all_sub_probs[g_name]
            sub_angle_step = 2 * np.pi / len(subs)
            
            for j, (sub, s_n_prob) in enumerate(subs.items()):
                # 그룹 중심 주변으로 퍼짐
                offset = 1.5 + (1.0 - s_n_prob) * 2.0
                sub_angle = j * sub_angle_step
                
                sx = center_pos[0] + offset * np.cos(sub_angle)
                sy = center_pos[1] + offset * np.sin(sub_angle)
                
                G.add_node(sub)
                pos[sub] = np.array([sx, sy])
                colors.append('#E0E0E0') # 회색 (확장된 노드)
                sizes.append(1000)
                labels[sub] = sub.replace('r/', '')
                
                G.add_edge(predicted_sub, sub, color='#DDDDDD', style='dashed')
        else:
            # 기본 모드: 그룹 노드 하나만 표시
            node_id = f"Group:\n{g_name}"
            G.add_node(node_id)
            pos[node_id] = center_pos
            colors.append('#90EE90') # 연두색
            sizes.append(3000 + n_g_prob * 1000)
            
            raw_gp = top_prob[g_name]
            labels[node_id] = f"{g_name}\n({raw_gp*100:.1f}%)"
            
            G.add_edge(predicted_sub, node_id, color='#DDDDDD', style='dashed')

    plt.figure(figsize=(12, 12))
    edges = G.edges()
    styles = [G[u][v].get('style', 'solid') for u,v in edges]
    e_colors = [G[u][v].get('color', 'gray') for u,v in edges]
    
    nx.draw_networkx_edges(G, pos, style=styles, edge_color=e_colors, alpha=0.5)
    nx.draw_networkx_nodes(G, pos, node_color=colors, node_size=sizes, edgecolors='black', alpha=0.9)
    
    # 레이블 (배경 박스)
    text_items = nx.draw_networkx_labels(G, pos, labels=labels, font_family='Malgun Gothic', font_size=9, font_weight='bold')
    for t in text_items.values():
        t.set_bbox(dict(facecolor='white', alpha=0.7, edgecolor='none', boxstyle='round,pad=0.2'))
        
    plt.title(f"서브레딧 관계성 맵 (중심: {predicted_sub})", fontsize=15)
    plt.xlim(-20, 20) # 범위 넉넉하게
    plt.ylim(-20, 20)
    plt.axis('off')
    plt.show()

# UI 실행
out = widgets.Output()
# 예측된 그룹을 제외한 다른 그룹들에 대한 체크박스 생성
other_group_names = [g for g in GROUP_MAP.keys() if g != predicted_group]
checkboxes = {g: widgets.Checkbox(False, description=f'{g} 그룹 펼치기') for g in other_group_names}

def update_plot(_):
    out.clear_output(wait=True)
    # 체크된 그룹 리스트 생성
    expanded = [g for g, box in checkboxes.items() if box.value]
    with out:
        draw_network(expanded)

# 이벤트 연결
for box in checkboxes.values():
    box.observe(update_plot, names='value')

print(f"입력 텍스트: {example_title}")
print("아래 체크박스를 클릭하여 다른 그룹의 상세 정보를 확인하세요.")
display(widgets.HBox(list(checkboxes.values())), out)

# 초기 화면 그리기
with out:
    draw_network([])

입력 텍스트: How to build a web app using Python and JavaScript
아래 체크박스를 클릭하여 다른 그룹의 상세 정보를 확인하세요.


Output()